### Getting Started 

#### Install Google Gen AI SDK for Python

In [1]:
%pip install --upgrade --quiet google-genai

Note: you may need to restart the kernel to use updated packages.


#### Restart runtime
To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

In [2]:
# restart the kernel after libraries are loaded
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

### Set Google Cloud project information and create client

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).



In [1]:
import os
# Define project information
PROJECT_ID = "qwiklabs-gcp-02-629a6ca913e4"  # @param {type:"string"}
LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "global")

# Create the API client
from google import genai
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

#### Import libraries


In [2]:
from google.genai.types import (
    FunctionDeclaration,
    GenerateContentConfig,
    Tool,
    Part
)

### Task 3. Create a function call using Gemini

In [15]:
# Task 3.1
# use the following documentation to assist you complete this cell
# https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/function-calling
# Load Gemini 2.5 Flash Model
model_id = ("gemini-2.5-flash")

In [16]:
# Task 3.2
# use the following documentation to assist you complete this cell
# https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/function-calling
get_current_weather_func = FunctionDeclaration(
    name="get_current_weather",
    description="Get the current weather in a given location",
    parameters={
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "Location"
            }
        }
    },
)

In [17]:
# Task 3.3
# use the following documentation to assist you complete this cell
# https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/function-calling
weather_tool = Tool(
    function_declarations=[get_current_weather_func],
)

In [18]:
# Task 3.4
# use the following documentation to assist you complete this cell
# https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/function-calling
prompt = "What is the weather like in Boston?"
response = client.models.generate_content(
    model=model_id,
    contents=prompt,
    config=GenerateContentConfig(
        tools=[weather_tool],
        temperature=0,
    ),
)
response

GenerateContentResponse(
  candidates=[
    Candidate(
      avg_logprobs=-0.43562361172267366,
      content=Content(
        parts=[
          Part(
            function_call=FunctionCall(
              args=<... Max depth ...>,
              name=<... Max depth ...>
            ),
            thought_signature=b"\n\xcb\x01\x01\xe3\xf1\xff^$ Z\x93\xef\xe8zi\x11l\xb8\x97\x8b:R\xcbPk\xee\xb1\xfbX\xc8@4\xb5\xe9\x8ac\xa5O\x96u/\xe20\xf3\xa2\x80;\x994\xfe-{\x8c\xcf\xef[\xbb\xe7ef\xcd[\xfa\x17[\xe1\xee?\x94\x7f\xc7x`\x08\xc4\xd8_'b5*?}l\xb7\x8f\xdd\xbd\xd9\n\xe9\xab\xc1o\x0c\x87...'
          ),
        ],
        role='model'
      ),
      finish_reason=<FinishReason.STOP: 'STOP'>
    ),
  ],
  create_time=datetime.datetime(2025, 11, 12, 7, 45, 22, 871111, tzinfo=TzInfo(0)),
  model_version='gemini-2.5-flash',
  response_id='EjsUaceVNeKCqsMPwbaR2AE',
  sdk_http_response=HttpResponse(
    headers=<dict len=9>
  ),
  usage_metadata=GenerateContentResponseUsageMetadata(
    candidates_token

### Task 4. Describe video contents using Gemini

In [9]:
# Run the following cell to import required libraries 
from google.genai.types import (
    GenerationConfig,
    Image,
    Part,
)

In [10]:
# Task 4.1
# Load the correct Gemini model use the following documentation to assist:
# https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/overview#supported-use-cases
# Load Gemini 2.5 Flash Model
multimodal_model = ("gemini-2.5-flash")

In [11]:
import http.client
import typing
import urllib.request

import IPython.display
from PIL import Image as PIL_Image
from PIL import ImageOps as PIL_ImageOps


def display_images(
    images: typing.Iterable[Image],
    max_width: int = 600,
    max_height: int = 350,
) -> None:
    for image in images:
        pil_image = typing.cast(PIL_Image.Image, image._pil_image)
        if pil_image.mode != "RGB":
            # RGB is supported by all Jupyter environments (e.g. RGBA is not yet)
            pil_image = pil_image.convert("RGB")
        image_width, image_height = pil_image.size
        if max_width < image_width or max_height < image_height:
            # Resize to display a smaller notebook image
            pil_image = PIL_ImageOps.contain(pil_image, (max_width, max_height))
        IPython.display.display(pil_image)


def get_image_bytes_from_url(image_url: str) -> bytes:
    with urllib.request.urlopen(image_url) as response:
        response = typing.cast(http.client.HTTPResponse, response)
        image_bytes = response.read()
    return image_bytes


def load_image_from_url(image_url: str) -> Image:
    image_bytes = get_image_bytes_from_url(image_url)
    return Image.from_bytes(image_bytes)


def display_content_as_image(content: str | Image | Part) -> bool:
    if not isinstance(content, Image):
        return False
    display_images([content])
    return True


def display_content_as_video(content: str | Image | Part) -> bool:
    if not isinstance(content, Part):
        return False
    part = typing.cast(Part, content)
    file_path = part.file_data.file_uri.removeprefix("gs://")
    video_url = f"https://storage.googleapis.com/{file_path}"
    IPython.display.display(IPython.display.Video(video_url, width=600))
    return True


def print_multimodal_prompt(contents: list[str | Image | Part]):
    """
    Given contents that would be sent to Gemini,
    output the full multimodal prompt for ease of readability.
    """
    for content in contents:
        if display_content_as_image(content):
            continue
        if display_content_as_video(content):
            continue
        print(content)

In [14]:
# Task 4.2 Generate a video description
# In this cell, update the prompt to ask Gemini to describe the video URL referenced.
# You can use the documentation at the following link to assist.
# https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/sdk-for-gemini/gemini-sdk-overview-reference#generate-content-from-video
# https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#sample-requests-text-stream-response
# Video URI: gs://github-repo/img/gemini/multimodality_usecases_overview/mediterraneansea.mp4

prompt = """
What is shown in this video?
Where should I go to see it?
What are the top 5 places in the world that look like this?
"""
video = Part.from_uri(
    file_uri="gs://github-repo/img/gemini/multimodality_usecases_overview/mediterraneansea.mp4",
    mime_type="video/mp4",
)
contents = [prompt, video]

responses = client.models.generate_content_stream(
    model=multimodal_model,
    contents=contents
)

print("-------Prompt--------")
print_multimodal_prompt(contents)

print("\n-------Response--------")
for response in responses:
    print(response.text, end="")

-------Prompt--------

What is shown in this video?
Where should I go to see it?
What are the top 5 places in the world that look like this?




-------Response--------
This video shows an aerial view of a vibrant **coastal city with a historical harbor nestled against dramatic cliffs**. The turquoise-blue sea is visible, with waves crashing against a stone breakwater that features a lighthouse. Inside the sheltered harbor, numerous boats, including yachts and what appears to be a tour boat, are docked. White and light-colored buildings of the city are spread across the high cliffs overlooking the sea.

You should go to **Antalya, Turkey** to see this specific location. This is the **Old Harbor (Kaleiçi Marina)** in Antalya, famous for its picturesque setting with the Taurus Mountains in the distance (though not clearly visible in this specific frame) and the city built along the towering cliffs known as the "Falezler."

Based on the combination of a coastal city built on cliffs, featuring a harbor or marina, and beautiful clear waters, here are 5 places in the world that look similar or offer a comparable experience:

1.  **S